# Experimental Pandas querying for Dashboard

In [232]:
from os import environ

import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
from psycopg2 import connect, extras
from psycopg2.extensions import connection
import altair as alt

load_dotenv()


SELECT_ALL_QUERY = """SELECT prices.price_id, prices.updated_at, prices.price, products.product_id, products.product_name, products.product_url, products.website_name, products.image_url, products.product_availability, users.user_id, users.email, users.first_name, users.last_name, subscriptions.subscription_id
                FROM prices
                JOIN products ON products.product_id = prices.product_id
                JOIN subscriptions ON subscriptions.product_id = products.product_id
                JOIN users ON users.user_id = subscriptions.user_id
                ORDER BY product_id, updated_at DESC;"""

COLUMNS = {"price_id": "Price ID", "updated_at": "Updated At", "price": "Price", "product_id": "Product ID", "product_name": "Product Name", "product_url": "Product URL", "image_url": "Image URL",
           "product_availability": "Product Availability", "website_name": "Website Name", "user_id": "User ID", "first_name": "User FirstName", "last_name": "User LastName", "email": "User Email", "subscription_id": "Subscription ID"}

## Making the Database connection

In [233]:
def get_database_connection() -> connection:
    """
    Return a connection our database.
    """
    try:
        return connect(
            user=environ["DB_USER"],
            password=environ["DB_PASSWORD"],
            host=environ["DB_HOST"],
            port=environ["DB_PORT"],
            database=environ["DB_NAME"]
        )
    except ConnectionError as error:
        return error


conn = get_database_connection()

### The Entire Merged Table

In [234]:
def load_all_database_info(db_conn: connection) -> pd.DataFrame:
    """Extract all data from the database."""

    with db_conn.cursor(cursor_factory=extras.RealDictCursor) as cur:

        cur.execute(SELECT_ALL_QUERY)

        result = cur.fetchall()

        return pd.DataFrame(result).rename(columns=COLUMNS)
    

all_info = load_all_database_info(conn)

all_info.head(20)

,Price ID,Updated At,Price,Product ID,Product Name,Product URL,Website Name,Image URL,Product Availability,User ID,User Email,User FirstName,User LastName,Subscription ID
0,408,2024-01-08 10:42:35.877799,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
1,393,2024-01-08 10:42:30.169145,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
2,378,2024-01-08 10:42:14.416786,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
3,363,2024-01-08 10:42:03.851748,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
4,348,2024-01-08 10:40:37.036354,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
5,333,2024-01-08 10:40:28.571707,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
6,318,2024-01-08 10:40:22.913115,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
7,303,2024-01-08 10:39:59.682905,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
8,288,2024-01-08 10:27:28.782287,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
9,273,2024-01-08 10:25:01.480036,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1


### Get the number of users

In [235]:
num_users = all_info["User Email"].nunique()

num_users

5

### Finding All Information of just 1 user

In [236]:
user_id_1_df = all_info[all_info['User ID'] == 1]

user_id_1_df

,Price ID,Updated At,Price,Product ID,Product Name,Product URL,Website Name,Image URL,Product Availability,User ID,User Email,User FirstName,User LastName,Subscription ID
0,408,2024-01-08 10:42:35.877799,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
1,393,2024-01-08 10:42:30.169145,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
2,378,2024-01-08 10:42:14.416786,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
3,363,2024-01-08 10:42:03.851748,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
4,348,2024-01-08 10:40:37.036354,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,300,2024-01-08 10:39:58.362209,52.0,15,Kyo The Brand sequin long sleeve cut out detai...,https://www.asos.com/kyo/kyo-the-brand-sequin-...,www.asos.com,https://images.asos-media.com/products/kyo-the...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,21
586,285,2024-01-08 10:27:27.786202,52.0,15,Kyo The Brand sequin long sleeve cut out detai...,https://www.asos.com/kyo/kyo-the-brand-sequin-...,www.asos.com,https://images.asos-media.com/products/kyo-the...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,21
587,270,2024-01-08 10:25:00.679127,52.0,15,Kyo The Brand sequin long sleeve cut out detai...,https://www.asos.com/kyo/kyo-the-brand-sequin-...,www.asos.com,https://images.asos-media.com/products/kyo-the...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,21
588,255,2024-01-08 10:24:40.494884,52.0,15,Kyo The Brand sequin long sleeve cut out detai...,https://www.asos.com/kyo/kyo-the-brand-sequin-...,www.asos.com,https://images.asos-media.com/products/kyo-the...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,21


### Finding the most recent price of each subscription for 1 user

In [237]:
sorted_df = user_id_1_df.sort_values(
    by=['Product ID', 'Updated At'], ascending=[True, False])

# Group by 'Product ID' and take the first (most recent) entry of each group
most_recent_prices = sorted_df.groupby('Product ID').first()

most_recent_prices

,Price ID,Updated At,Price,Product Name,Product URL,Website Name,Image URL,Product Availability,User ID,User Email,User FirstName,User LastName,Subscription ID
Product ID,,,,,,,,,,,,,
1,408,2024-01-08 10:42:35.877799,22.0,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
2,395,2024-01-08 10:42:34.436034,22.0,ASOS DESIGN Hourglass slim mom jeans in washed...,https://www.asos.com/asos-design/asos-design-h...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,2
3,396,2024-01-08 10:42:34.436152,24.0,Clinique Almost Lipstick - Pink Honey,https://www.asos.com/clinique/clinique-almost-...,www.asos.com,https://images.asos-media.com/products/cliniqu...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,3
6,399,2024-01-08 10:42:34.436636,80.0,Miss Selfridge Premium embellished star puff s...,https://www.asos.com/miss-selfridge/miss-selfr...,www.asos.com,https://images.asos-media.com/products/miss-se...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,8
7,412,2024-01-08 16:21:08.225490,35.0,River Island zebra print wrap midi dress in li...,https://www.asos.com/river-island/river-island...,www.asos.com,https://images.asos-media.com/products/river-i...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,18
9,402,2024-01-08 10:42:34.438499,28.0,Elemis Pro-Collagen Green Fig Cleansing Balm 50g,https://www.asos.com/elemis/elemis-pro-collage...,www.asos.com,https://images.asos-media.com/products/elemis-...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,12
10,407,2024-01-08 10:42:35.399912,22.99,Stradivarius tailored straight trouser in dark...,https://www.asos.com/stradivarius/stradivarius...,www.asos.com,https://images.asos-media.com/products/stradiv...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,14
11,409,2024-01-08 10:42:35.891054,35.99,Stradivarius STR baggy jeans in vintage wash,https://www.asos.com/stradivarius/stradivarius...,www.asos.com,https://images.asos-media.com/products/stradiv...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,15
12,403,2024-01-08 10:42:34.438899,25.0,Monki asymmetric ruffle detail midi dress in red,https://www.asos.com/monki/monki-asymmetric-ru...,www.asos.com,https://images.asos-media.com/products/monki-a...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,16


### Get the names of all the products a user is subscribed to

In [238]:
user_id_1_df["Product Name"].unique()

array(['ASOS DESIGN off shoulder twist midi dress with long sleeves in black',
       'ASOS DESIGN Hourglass slim mom jeans in washed black',
       'Clinique Almost Lipstick - Pink Honey',
       'Miss Selfridge Premium embellished star puff sleeve mini dress with tie back detail in black - BLACK',
       'River Island zebra print wrap midi dress in light pink',
       'Elemis Pro-Collagen Green Fig Cleansing Balm 50g',
       'Stradivarius tailored straight trouser in dark grey ',
       'Stradivarius STR baggy jeans in vintage wash',
       'Monki asymmetric ruffle detail midi dress in red',
       'My Accessories London touch screen knitted gloves in black ',
       'ASOS DESIGN zip through cardigan with collar in charcoal',
       'Kyo The Brand sequin long sleeve cut out detail maxi dress in red'],
      dtype=object)

### Get the Total number of Products a user is subscribed to

In [239]:
len(user_id_1_df["Product Name"].unique())

12

### Group by Product Name

In [240]:
grouped = all_info.groupby('Product Name')

grouped.head()

,Price ID,Updated At,Price,Product ID,Product Name,Product URL,Website Name,Image URL,Product Availability,User ID,User Email,User FirstName,User LastName,Subscription ID
0,408,2024-01-08 10:42:35.877799,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
1,393,2024-01-08 10:42:30.169145,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
2,378,2024-01-08 10:42:14.416786,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
3,363,2024-01-08 10:42:03.851748,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
4,348,2024-01-08 10:40:37.036354,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,390,2024-01-08 10:42:28.000747,52.0,15,Kyo The Brand sequin long sleeve cut out detai...,https://www.asos.com/kyo/kyo-the-brand-sequin-...,www.asos.com,https://images.asos-media.com/products/kyo-the...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,21
580,375,2024-01-08 10:42:13.471751,52.0,15,Kyo The Brand sequin long sleeve cut out detai...,https://www.asos.com/kyo/kyo-the-brand-sequin-...,www.asos.com,https://images.asos-media.com/products/kyo-the...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,21
581,360,2024-01-08 10:42:02.577364,52.0,15,Kyo The Brand sequin long sleeve cut out detai...,https://www.asos.com/kyo/kyo-the-brand-sequin-...,www.asos.com,https://images.asos-media.com/products/kyo-the...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,21
582,345,2024-01-08 10:40:35.877649,52.0,15,Kyo The Brand sequin long sleeve cut out detai...,https://www.asos.com/kyo/kyo-the-brand-sequin-...,www.asos.com,https://images.asos-media.com/products/kyo-the...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,21


### Get the most recent price of each product

In [241]:
most_recent_prices = grouped.apply(lambda x: x.sort_values(
    by='Updated At', ascending=False).head(1))

most_recent_prices = most_recent_prices.reset_index(drop=True)

most_recent_prices

,Price ID,Updated At,Price,Product ID,Product Name,Product URL,Website Name,Image URL,Product Availability,User ID,User Email,User FirstName,User LastName,Subscription ID
0,395,2024-01-08 10:42:34.436034,22.0,2,ASOS DESIGN Hourglass slim mom jeans in washed...,https://www.asos.com/asos-design/asos-design-h...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,2
1,408,2024-01-08 10:42:35.877799,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
2,406,2024-01-08 10:42:34.440497,36.0,14,ASOS DESIGN zip through cardigan with collar i...,https://www.asos.com/asos-design/asos-design-z...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,20
3,396,2024-01-08 10:42:34.436152,24.0,3,Clinique Almost Lipstick - Pink Honey,https://www.asos.com/clinique/clinique-almost-...,www.asos.com,https://images.asos-media.com/products/cliniqu...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,3
4,398,2024-01-08 10:42:34.436395,94.5,5,Columbia Puffect II puffer coat in khaki,https://www.asos.com/columbia/columbia-puffect...,www.asos.com,https://images.asos-media.com/products/columbi...,True,2,trainee.zander.snow@sigmalabs.co.uk,Zander,Snow,7
5,402,2024-01-08 10:42:34.438499,28.0,9,Elemis Pro-Collagen Green Fig Cleansing Balm 50g,https://www.asos.com/elemis/elemis-pro-collage...,www.asos.com,https://images.asos-media.com/products/elemis-...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,12
6,405,2024-01-08 10:42:34.439985,52.0,15,Kyo The Brand sequin long sleeve cut out detai...,https://www.asos.com/kyo/kyo-the-brand-sequin-...,www.asos.com,https://images.asos-media.com/products/kyo-the...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,21
7,410,2024-01-08 16:06:54.500258,60.0,16,Levi's sweatshirt with batwing logo in green,https://www.asos.com/levis/levis-sweatshirt-wi...,www.asos.com,https://images.asos-media.com/products/levis-s...,True,2,trainee.zander.snow@sigmalabs.co.uk,Zander,Snow,22
8,399,2024-01-08 10:42:34.436636,80.0,6,Miss Selfridge Premium embellished star puff s...,https://www.asos.com/miss-selfridge/miss-selfr...,www.asos.com,https://images.asos-media.com/products/miss-se...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,8
9,403,2024-01-08 10:42:34.438899,25.0,12,Monki asymmetric ruffle detail midi dress in red,https://www.asos.com/monki/monki-asymmetric-ru...,www.asos.com,https://images.asos-media.com/products/monki-a...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,16


In [242]:
most_recent_prices["Price (£)"] = most_recent_prices["Price"].astype(float)

latest_data = most_recent_prices[["Product Name", "Price (£)"]]

latest_price_readings = alt.Chart(latest_data).mark_bar().encode(
    y=alt.Y('Product Name:N'),
    x=alt.X('Price (£):Q'),
    tooltip=['Product Name:N', 'Price (£):Q'],
    color=alt.Color('Product Name:N', legend=None).scale(scheme='blues')
).properties(
    title='Latest Price of Products',
    width=600
)

latest_price_readings

alt.Chart(...)

### Get Popularity of each product (based on Number of subscriptions per product)

In [243]:
grouped = all_info.groupby('Product Name')
product_popularity = grouped['Subscription ID'].nunique()

product_popularity = product_popularity.reset_index()

product_popularity.columns = ['Product Name', 'Popularity']


product_popularity

,Product Name,Popularity
0,ASOS DESIGN Hourglass slim mom jeans in washed...,1
1,ASOS DESIGN off shoulder twist midi dress with...,1
2,ASOS DESIGN zip through cardigan with collar i...,1
3,Clinique Almost Lipstick - Pink Honey,2
4,Columbia Puffect II puffer coat in khaki,1
5,Elemis Pro-Collagen Green Fig Cleansing Balm 50g,1
6,Kyo The Brand sequin long sleeve cut out detai...,1
7,Levi's sweatshirt with batwing logo in green,1
8,Miss Selfridge Premium embellished star puff s...,1
9,Monki asymmetric ruffle detail midi dress in red,1


### Get price over time of all products

In [244]:
all_info['Updated At'] = pd.to_datetime(all_info['Updated At'])

all_info['Price'] = all_info['Price'].astype(float)

    # Create the Altair line chart
line_chart = alt.Chart(all_info).mark_line().encode(
        x=alt.X('Updated At:T', axis=alt.Axis(title='Time')),
        y=alt.Y('Price:Q', axis=alt.Axis(title='Price')),
        color='Product Name:N',
        tooltip=['Product Name:N', 'Price:Q', 'Updated At:T']
    ).properties(
        title='Price Over Time by Product',
        width=600,
        height=400
    )

line_chart

alt.Chart(...)

### Get Number of Subscriptions over time